# Observatório Imobiliário de Araraquara
## Análise Exploratória de Dados

Este notebook demonstra como explorar os dados coletados do mercado imobiliário de Araraquara.

In [1]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Carregar Dados

In [2]:
# Listar arquivos disponíveis
data_dir = Path('../data/raw')
arquivos = list(data_dir.glob('*.csv'))

print("Arquivos disponíveis:")
for i, arquivo in enumerate(arquivos):
    print(f"{i+1}. {arquivo.name}")

Arquivos disponíveis:


In [ ]:
# Carregar arquivo (ajuste o índice conforme necessário)
if arquivos:
    df = pd.read_csv(arquivos[0])
    print(f"Dados carregados: {len(df)} registros")
    df.head()

## 2. Exploração Inicial

In [ ]:
# Informações gerais
print("Informações do dataset:")
df.info()

In [ ]:
# Estatísticas descritivas
df.describe()

In [ ]:
# Valores ausentes
print("Valores ausentes por coluna:")
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

## 3. Análise de Preços

In [ ]:
# Distribuição de preços
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
df['preco'].hist(bins=50, ax=axes[0], edgecolor='black')
axes[0].set_xlabel('Preço (R$)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição de Preços')

# Boxplot
df.boxplot(column='preco', ax=axes[1])
axes[1].set_ylabel('Preço (R$)')
axes[1].set_title('Boxplot de Preços')

plt.tight_layout()
plt.show()

In [ ]:
# Estatísticas de preço
print("Estatísticas de Preço:")
print(f"Média: R$ {df['preco'].mean():,.2f}")
print(f"Mediana: R$ {df['preco'].median():,.2f}")
print(f"Mínimo: R$ {df['preco'].min():,.2f}")
print(f"Máximo: R$ {df['preco'].max():,.2f}")
print(f"Amplitude: {df['preco'].max() / df['preco'].min():.2f}x")

## 4. Análise por Tipo de Imóvel

In [ ]:
# Distribuição por tipo
if 'tipo_imovel' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Contagem
    tipo_counts = df['tipo_imovel'].value_counts()
    tipo_counts.plot(kind='bar', ax=axes[0])
    axes[0].set_xlabel('Tipo de Imóvel')
    axes[0].set_ylabel('Quantidade')
    axes[0].set_title('Quantidade por Tipo de Imóvel')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Preço médio
    preco_por_tipo = df.groupby('tipo_imovel')['preco'].mean().sort_values()
    preco_por_tipo.plot(kind='barh', ax=axes[1])
    axes[1].set_xlabel('Preço Médio (R$)')
    axes[1].set_ylabel('Tipo de Imóvel')
    axes[1].set_title('Preço Médio por Tipo de Imóvel')
    
    plt.tight_layout()
    plt.show()

## 5. Análise por Bairro

In [ ]:
# Top 10 bairros
if 'bairro' in df.columns:
    top_bairros = df['bairro'].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    top_bairros.plot(kind='barh')
    plt.xlabel('Quantidade de Anúncios')
    plt.ylabel('Bairro')
    plt.title('Top 10 Bairros com Mais Anúncios')
    plt.tight_layout()
    plt.show()

In [ ]:
# Preço médio por bairro (top 10)
if 'bairro' in df.columns:
    # Filtrar apenas bairros com pelo menos 5 anúncios
    bairros_relevantes = df['bairro'].value_counts()[df['bairro'].value_counts() >= 5].index
    df_filtrado = df[df['bairro'].isin(bairros_relevantes)]
    
    preco_bairro = df_filtrado.groupby('bairro')['preco'].mean().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    preco_bairro.plot(kind='barh')
    plt.xlabel('Preço Médio (R$)')
    plt.ylabel('Bairro')
    plt.title('Top 10 Bairros - Preço Médio (min. 5 anúncios)')
    plt.tight_layout()
    plt.show()

## 6. Análise de Amplitude (Desigualdade)

In [ ]:
# Calcular amplitude por bairro
if 'bairro' in df.columns:
    amplitude_bairro = df.groupby('bairro')['preco'].agg(['min', 'max', 'count'])
    amplitude_bairro['amplitude'] = amplitude_bairro['max'] / amplitude_bairro['min']
    
    # Filtrar bairros com pelo menos 10 anúncios
    amplitude_bairro = amplitude_bairro[amplitude_bairro['count'] >= 10]
    amplitude_bairro = amplitude_bairro.sort_values('amplitude', ascending=False).head(10)
    
    print("Top 10 Bairros - Maior Amplitude de Preços:")
    print(amplitude_bairro[['min', 'max', 'amplitude']])

## 7. Correlações

In [ ]:
# Selecionar colunas numéricas
colunas_numericas = ['preco', 'area', 'quartos', 'banheiros', 'vagas']
colunas_disponiveis = [col for col in colunas_numericas if col in df.columns]

if len(colunas_disponiveis) > 1:
    # Matriz de correlação
    correlacao = df[colunas_disponiveis].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlacao, annot=True, cmap='coolwarm', center=0)
    plt.title('Matriz de Correlação')
    plt.tight_layout()
    plt.show()

## 8. Salvar Análises

In [ ]:
# Criar resumo e salvar
resumo = {
    'total_imoveis': len(df),
    'preco_medio': df['preco'].mean(),
    'preco_mediano': df['preco'].median(),
    'amplitude_preco': df['preco'].max() / df['preco'].min()
}

print("Resumo da Análise:")
for chave, valor in resumo.items():
    print(f"{chave}: {valor}")